In [1]:
import amazon_reviews
import openai
import json
import os

openai.api_key  = '...'

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]


In [2]:
amazon, amazon_value = amazon_reviews.search()

b = amazon_value

json_file = amazon

with open(json_file, "r") as file:
    data = json.load(file)

reviews = data[0]


In [3]:
vector = []

for i, review in enumerate(reviews):

    text = review["body"]

    vector.append(text)

prompt = f"""

Analyze the following collection of reviews and employ topic modeling techniques to categorize the feedback into specific features of the product.
Divide each feature in positive characteristics and in negative characteristics.
Response format: 
                Feature:       
                    -Name: x
                    -Positive characteristics: y
                    -Negative characteristics: z
                    -If there are no positive or negative characteristics, write "Not applicable".

Provide it in JSON format to save in JSON file.

Review text: '''{vector}'''
"""
response = get_completion(prompt)

# Save response to a JSON file
with open('response.json', 'w') as f:
    f.write(response)

#print(response, "\n")

In [5]:
import json
import dominate
from dominate.tags import *

# Load your JSON data
with open('response.json', 'r') as f:
    data = json.load(f)

doc = dominate.document(title='Features')

with doc.head:
    style("""
        body {
            font-family: Arial, sans-serif;
        }
        .feature {
            border: 1px solid #ddd;
            margin-bottom: 10px;
            padding: 10px;
            border-radius: 5px;
        }
        .feature-name {
            font-size: 20px;
            color: #444;
        }
        .positive {
            color: green;
        }
        .negative {
            color: red;
        }
    """)

with doc:
    h1(f'Product ID: {amazon_value}', style="text-align:center;")

    for feature in data['features']:
        with div(cls='feature'):
            h2(feature['name'], cls='feature-name')
            with div(cls='positive'):
                h3('Positive characteristics:')
                ul(li(pc) for pc in [feature['positive_characteristics']])   # Updated this line
            with div(cls='negative'):
                h3('Negative characteristics:')
                ul(li(nc) for nc in [feature['negative_characteristics']])   # Updated this line


with open('output.html', 'w') as f:
    f.write(str(doc))